# Machine Learning Group Project 

User game rating prediction & systematic discount offering on Steam. Project developed by Team XX composed by:
| Student Name | Student Number | Class Group |
| --- | --- | --- |
| **Alessandro Maugeri** | 53067 | TA |
| **Frank Andreas Bauer** | XXXX | XX |
|  **Johannes Rahn** | XXXX | XX |
| **Nicole Zoppi** | XXXX | XX |
| **Yannick von der Heyden** | XXXX | XX |

## Importing Packages

In [42]:
import ast
import pandas as pd
import numpy as np
from datetime import datetime

## Importing Data

The data for this project was retrieved from [Kaggle](https://www.kaggle.com/datasets/antonkozyriev/game-recommendations-on-steam?select=games.csv) and stored in the "data" folder found in the notebook's directory. The folder includes **five data files**:

The CSV file **[games.csv](data/games.csv)** presents data concerning individual games in the Steam library:

| Column | Description | Example|
| --- | --- | --- |
| **app_id** | Product ID on Steam | 620 |
| **title** | Product Commercial Title | Portal 2|
|  **date_release** | Release Date of Title (y-m-d) | 2011-04-18 |
| **win** | Boolean Denoting Compatibility to Windows Computers | True |
| **mac** | Boolean Denoting Compatibility to Mac Computers  | True | 
| **linux** | Boolean Denoting Compatibility to Linux Computers  | True |
| **rating** | Categorical Rating of Product (e.g. "Positive")| Overwhelmingly Positive |
| **positive_ratio** | Ratio of Postive Feedback for Game  | 98 |
| **user_reviews** | Number of Reviews  | 267142 |
| **price_final** | Final Price in USD | 9.99 |
| **price_original** | Price Before Discounts in USD | 9.99 |
| **discount** | Applied Discount | 0 |
| **steam_deck** | Compatible with [Steam Deck](https://store.steampowered.com/steamdeck)? | True |



In [43]:
df_games_data = pd.read_csv("data/games.csv")
df_games_data.head(2)

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
0,10090,Call of Duty: World at War,2008-11-18,True,False,False,Very Positive,92,37039,19.99,19.99,0.0,True
1,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,9.99,0.0,True


----
The **CSV file [users.csv](data/users.csv)** presents data concerning individual users found in the datasets:

| Column | Description | Example|
| --- | --- | --- |
| **user_id** | User ID on Steam | 5693478 |
| **products** | Number of Products from Steam Library Owned | 156 |
|  **reviews** | Number of Reviews Published | 1 |

In [44]:
df_users = pd.read_csv("data/users.csv")
df_users.head(2)

,user_id,products,reviews
0,5693478,156,1
1,3595958,329,3


----
The **CSV file [recommendations.csv](data/recommendations.csv)** has a many-to-many relationship to both users.csv and games.csv and contains data concerning user reviews of specific games:

| Column | Description | Example|
| --- | --- | --- |
| **app_id** | Product ID on Steam | 620 |
| **helpful** | Number of Users Who Found Review Helpful | 0 |
|  **funny** | Number of Users Who Found Review Funny | 0 |
| **date** | Date in Which Review was Published (y-m-d) | 2022-12-12 |
| **is_recommended** | Does the User Recommend the Title | True | 
| **hours** | Hours Spent by User Playing Game  | 36.3 |
| **user_id** | User ID of Review Author | 19954 |
| **review_id** | ID of Individual Review  | 0 |

In [45]:
df_recommendations = pd.read_csv("data/recommendations.csv")
df_recommendations.head(2)

,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id
0,975370,0,0,2022-12-12,True,36.3,19954,0
1,304390,4,0,2017-02-17,False,11.5,1098,1


----
Finally, the folder includes a **JSON file [games_metadata.json](data/games_metadata.json)** containing metadata on individual games:

| Column | Description | Example|
| --- | --- | --- |
| **app_id** | Product ID on Steam | 304430 |
| **description** | Game Description on Steam | "Hunted and alone, a boy finds himself drawn into the center of a dark project. INSIDE is a dark, narrative-driven platformer combining intense action with challenging puzzles. It has been critically acclaimed for its moody art style, ambient soundtrack and unsettling atmosphere." |
|  **tags** | Additional Tags on Steam Platform | ["2.5D", "Story Rich", "Puzzle Platformer" , "Atmospheric" , "Adventure" , "Indie" , "Dark" , "Horror" , "Singleplayer" , "Action-Adventure" , "Puzzle" , "Multiple Endings" , "Exploration" , "2D Platformer" , "Platformer" , "Controller" , "Soundtrack" , "Ambient" , "Action" , "Narrative"] |

In [46]:
df_games_meta_data = pd.read_json('data/games_metadata.json', lines=True)
df_games_meta_data.head(2)

,app_id,description,tags
0,10090,"Call of Duty is back, redefining war like you'...","[Zombies, World War II, FPS, Multiplayer, Acti..."
1,13500,Enter the dark underworld of Prince of Persia ...,"[Action, Adventure, Parkour, Third Person, Gre..."


----
Two additional data files are utilized from a separate [Kaggle page](https://www.kaggle.com/datasets/nikdavis/steam-store-games?select=steam.csv) to enrich the analysis. These are imported below.


The **CSV file [steam.csv](data/steam.csv)** provides additional data on the games:

| Column | Description | Example|
| --- | --- | --- |
| **appid** | Product ID on Steam| 10 |
| **name** | Name of Game | Counter-Strike |
|  **release_date** | Release Date of Title (y-m-d) | 2000-11-01 |
|  **english** | Is the Game Available in English? | 1 |
|  **developer** | Developer Company of Game | Valve |
|  **publisher** | Publishing Company of Game | Valve |
|  **platforms** | Semicolon-Delimited List of Systems that Can Run the Game | windows;mac;linux |
|  **required_age** | Minimum Age Required to Play by PEGI UK (0 Means Unsupplied) | 0 |
|  **categories** | Game Categorisation (Semicolon Delimited) | Multi-player;Online Multi-Player... |
|  **genres** | Game's Genre (Semicolon Delimteted) | Action |
|  **steamspy_tags** | Tags from [Steamspy API](https://steamspy.com/) (Semicolon Delimited) | Action;FPS;Multiplayer |
|  **achievements** | Number of In-Game Achievements (If Any) | 0 |
|  **positive_ratings** | Number of Positive Ratings | 124534 |
|  **negative_ratings** | Number of Negative Ratings | 3339 |
|  **average_playtime** | Average Playtime by User in Minutes| 17612 |
|  **median_playtime** | Median Playtime by User in Minutes | 317 |
|  **owners** | Number of Users that Own the Game (Bracket) | 10000000-20000000 |
|  **price** | Price of Game | 7.19 |



In [47]:
df_games_additional = pd.read_csv("data/steam.csv")
df_games_additional.head(2)

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99


### JOEY'S TASK
The **CSV file [blabla.csv](data/steam.csv)** contains information about system requirements in order to run the games:

| Column | Description | Example|
| --- | --- | --- |
| **JOEY** | Product ID on Steam| 10 |
| **JOEY** | Name of Game | Counter-Strike |
|  **JOEY** | Release Date of Title (y-m-d) | 2000-11-01 |
|  **JOEY** | Is the Game Available in English? | 1 |

In [48]:
# File Import 

## Preliminary Data Exploration

In [49]:
df_games_data["rating"].unique()

array(['Very Positive', 'Positive', 'Mixed', 'Mostly Positive',
       'Overwhelmingly Positive', 'Mostly Negative',
       'Overwhelmingly Negative', 'Negative', 'Very Negative'],
      dtype=object)

In [50]:
df_games_additional["owners"].unique()

array(['10000000-20000000', '5000000-10000000', '2000000-5000000',
       '20000000-50000000', '100000000-200000000', '50000000-100000000',
       '20000-50000', '500000-1000000', '100000-200000', '50000-100000',
       '1000000-2000000', '200000-500000', '0-20000'], dtype=object)

In [51]:
df_games_additional["developer"].value_counts()

Choice of Games               94
KOEI TECMO GAMES CO., LTD.    72
Ripknot Systems               62
Laush Dmitriy Sergeevich      51
Nikita "Ghost_RUS"            50
                              ..
CRAPPY ZOMBIE GAME STUDIO      1
Ramon Mujica                   1
Oomst Games                    1
Joe Censored Games             1
Adept Studios GD               1
Name: developer, Length: 17113, dtype: int64

## Data Preparation

#### Games Data
- Turning relevant columns to datetime objects
- **AA IF ANYTHING ELSE PLEASE ADD**

In [52]:
# Turn date_release column to Pandas DateTime
df_games_data["date_release"] = pd.to_datetime(df_games_data["date_release"])

#### Additional Game Data
- Drop features that are redundant with df_games_data

In [53]:
# Drop Redundant Columns
df_games_additional.drop(["name","platforms", "release_date", "price", 
                          "positive_ratings", "negative_ratings", "categories"], axis = 1, inplace = True)

In [54]:
df_games_additional.head(1)

,appid,english,developer,publisher,required_age,genres,steamspy_tags,achievements,average_playtime,median_playtime,owners
0,10,1,Valve,Valve,0,Action,Action;FPS;Multiplayer,0,17612,317,10000000-20000000


#### Recommendations Data
- Turn date features into datetime objects
- Rename is_recommended to y, to signal it is the target variable

In [55]:
# Turn to DateTime
df_recommendations["date"] = pd.to_datetime(df_recommendations["date"])

# Rename column
df_recommendations.rename(columns = {"is_recommended": "y"}, inplace = True)

#### Games Metadata

In [56]:
# Turn the Tags Column Into a List
df_games_meta_data["tags"] = df_games_meta_data["tags"].astype(str).apply(ast.literal_eval)

## Merging Datasets

We merge the data into one single final DataFrame.

In [57]:
# Merge all information on games to one DataFrame
games_df = df_games_data.merge(df_games_meta_data, how = "inner")
games_df = games_df.merge(df_games_additional, how = "inner",
                          left_on = "app_id", right_on = "appid")

# Merge game information into the recommendations DataFrame
recs_df = df_recommendations.merge(games_df, how = "inner", on = "app_id")

# Merge all information on users into a final DataFrame
final_df = recs_df.merge(df_users, how="inner", on = "user_id")

In [58]:
final_df.describe()

,app_id,helpful,funny,hours,user_id,review_id,positive_ratio,user_reviews,price_final,price_original,discount,appid,english,required_age,achievements,average_playtime,median_playtime,products,reviews
count,6.564923e+06,6.564923e+06,6.564923e+06,6.564923e+06,6.564923e+06,6.564923e+06,6.564923e+06,6.564923e+06,6.564923e+06,6.564923e+06,6.564923e+06,6.564923e+06,6564923.0,6.564923e+06,6.564923e+06,6.564923e+06,6.564923e+06,6.564923e+06,6.564923e+06
mean,3.331034e+05,3.460131e+00,1.394521e+00,1.904277e+02,3.004128e+06,5.136969e+06,8.782816e+01,4.158554e+05,2.118282e+01,2.241527e+01,4.317842e+00,3.331034e+05,1.0,4.369392e+00,7.823956e+01,4.590540e+03,1.782105e+03,1.909035e+02,3.940238e+00
std,1.851971e+05,5.834303e+01,4.805631e+01,2.373007e+02,1.650810e+06,2.967649e+06,7.836649e+00,1.132509e+06,1.556509e+01,1.650911e+01,1.751457e+01,1.851971e+05,0.0,7.590032e+00,1.401946e+02,5.843187e+03,1.969718e+03,3.690130e+02,5.721762e+00
min,7.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,4.400000e+01,1.740000e+02,0.000000e+00,0.000000e+00,0.000000e+00,7.000000e+01,1.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
25%,2.389600e+05,0.000000e+00,0.000000e+00,2.360000e+01,1.666029e+06,2.559700e+06,8.300000e+01,5.222600e+04,9.990000e+00,9.990000e+00,0.000000e+00,2.389600e+05,1.0,0.000000e+00,7.000000e+00,1.084000e+03,4.920000e+02,3.700000e+01,1.000000e+00
50%,2.941000e+05,0.000000e+00,0.000000e+00,8.300000e+01,3.047610e+06,5.120629e+06,8.800000e+01,1.056140e+05,1.999000e+01,1.999000e+01,0.000000e+00,2.941000e+05,1.0,0.000000e+00,4.300000e+01,2.086000e+03,1.147000e+03,9.500000e+01,2.000000e+00
75%,4.319600e+05,0.000000e+00,0.000000e+00,2.689000e+02,4.435053e+06,7.756638e+06,9.400000e+01,2.048400e+05,2.999000e+01,2.999000e+01,0.000000e+00,4.319600e+05,1.0,0.000000e+00,8.800000e+01,5.017000e+03,2.234000e+03,2.160000e+02,4.000000e+00
max,1.032430e+06,2.962700e+04,2.808800e+04,9.999000e+02,5.810792e+06,1.007227e+07,9.800000e+01,6.941137e+06,5.999000e+01,1.099900e+02,9.000000e+01,1.032430e+06,1.0,1.800000e+01,1.130000e+03,2.889700e+04,1.243400e+04,2.930800e+04,1.510000e+02


## Feature Engineering

**Elapsed Time:** A new feature which tracks the amount of time that has elapsed between the game's release and the review being logged. This could be interested because people who purchase a game right after its release are likely to be larger fans of the genre or franchise.

In [59]:
final_df["elapsed_time"] = final_df["date"] - final_df["date_release"]

**Relative Recommendation:** Find the rate at which users rated a game positively (out of the count of games they have rated). Then, find for each review whether a recommendation is in line with user's typical reviews or not.

In [60]:
# Count the number of reviews per user and rename column
usr_rating_count = final_df[["user_id","y"]].groupby("user_id").count()
usr_rating_count.rename(columns = {"y":"review_count"}, inplace = True)

# Find the number of positive reviews
usr_rating_count["positive_reviews"] = final_df[["user_id","y"]].groupby("user_id").sum()

# Find how many reviews out of total are positive
usr_rating_count["positive_rate"] = usr_rating_count["positive_reviews"] / usr_rating_count["review_count"]

In [61]:
# Merge review_count and positive_rate to the final_df
final_df = final_df.merge(usr_rating_count.reset_index()[["user_id", "review_count", "positive_rate"]], 
                          how = "left", on = "user_id")

# Use these to create an ulterior feature with recommendation relative to average recommendation
final_df["relaitve_rec"] = (final_df["y"] - final_df["positive_rate"])

**Tendency to Review:** People who review more often tend to...

In [82]:
# Find Review Tendency
final_df["review_tendency"] =  final_df["reviews"] /final_df["products"] 

# Find Tendency Relative to Median Because of Low Values
final_df["rel_review_tendency"] = (final_df["review_tendency"] - final_df["review_tendency"].median())/ final_df["review_tendency"].median()

**Positive & Negative Reviews:** Here we calculate the number of positive and negative reviews for each game. This is done through the positive_ratio column.

In [62]:
final_df["positive_reviews"] = final_df["positive_ratio"] * final_df["user_reviews"]
final_df["negative_reviews"] = (1 - final_df["positive_ratio"]) * final_df["user_reviews"]

In [64]:
final_df.head(2)

,app_id,helpful,funny,date,y,hours,user_id,review_id,title,date_release,...,median_playtime,owners,products,reviews,elapsed_time,review_count,positive_rate,relaitve_rec,positive_reviews,negative_reviews
0,304390,4,0,2017-02-17,False,11.5,1098,1,FOR HONOR™,2017-02-13,...,696,2000000-5000000,269,1,4 days,1,0.0,0.0,5172828,-5096757
1,304390,2,0,2017-03-23,False,180.6,140776,26,FOR HONOR™,2017-02-13,...,696,2000000-5000000,110,2,38 days,2,0.0,0.0,5172828,-5096757


## Final Dataset Exploration

In [65]:
# We see how many positive and negative recommendations there are
print(df_recommendations["y"].value_counts())

# What is their ratio?
print("\nHow many positive reviews for each negative one?")
df_recommendations["y"].value_counts()[0]/df_recommendations["y"].value_counts()[1]

True     8599822
False    1472448
Name: y, dtype: int64

How many positive reviews for each negative one?


5.840492839135915

## Model Preparation

In [24]:
# train_test_split with validation set

#### Logistic Regression Model

In [25]:
# Consider balancing out data because of overwhelming positivity

In [26]:
test = games_df = games_df.merge(df_games_additional, how = "inner",
                          left_on = "app_id", right_on = "appid")